In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
layer = keras.layers
print("tensorflow version check : ", tf.__version__)
print("gpu check", tf.test.is_gpu_available())


# 1. get dataset on memory

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("mnist dataset on memory")
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# 2. model definition

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
feature = layer.Conv2D(32, 3,activation='relu')(inputs)
feature = layer.MaxPool2D(pool_size=(2, 2))(feature)
feature = layer.Conv2D(64, 3, activation='relu')(feature)
feature = layer.MaxPool2D(pool_size=(2, 2))(feature)
flatten = layer.Flatten()(feature)
embedding = layer.Dense(128, activation='relu')(flatten)
prob = layer.Dense(10, activation='softmax')(embedding)
model = keras.Model(inputs, prob)

In [ ]:
model.summary()

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
print('========================Training===============================')

model.fit(x_train, y_train, epochs=5)

print('========================Evaluation===============================')

test_loss, test_acc = model.evaluate(x_test, y_test)

# del model
# keras.backend.clear_session()

# 3. Noise image problem?

In [ ]:
[fashine_train_x, fashine_train_y], [fashine_test_x, fashine_test_y] = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
def prepare_test_image(image):
  image = image[tf.newaxis, :, : , tf.newaxis] # [1, 28, 28, 1]
  return image / 255.

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(fashine_train_x[0], cmap='gray')
test_data = prepare_test_image(fashine_train_x[0])

In [ ]:
model.predict(test_data)

# I will show you a modified model for solving this!

In [ ]:
# 아웃풋 레이어의 activation은 None으로 해주고, sparse_categorical_crossentropy의 from_logits=True 옵션을 사용해줍니다.
# 그 이유는 선형분류기가 출력하는 score의 부호를 검사해서, 학습되지 않은 클래스의 이미지는 걸러주기 위해서에요.

inputs = keras.Input(shape=(28, 28, 1))
feature = layer.Conv2D(32, 3,activation='relu')(inputs)
feature = layer.MaxPool2D(pool_size=(2, 2))(feature)
feature = layer.Conv2D(64, 3, activation='relu')(feature)
feature = layer.MaxPool2D(pool_size=(2, 2))(feature)
flatten = layer.Flatten()(feature)
embedding = layer.Dense(128, activation='relu')(flatten)
scores = layer.Dense(10, activation=None)(embedding)
model = keras.Model(inputs, scores)

In [ ]:
model.summary()

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
print('========================Training===============================')

model.fit(x_train, y_train, epochs=5)

print('========================Evaluation===============================')

test_loss, test_acc = model.evaluate(x_test, y_test)

### 아래 코드를 실행해서 기존 mnist와 상관 없는 fashine mnist를 넣어 테스트를 해봅니다.
### score 값이 모두 큰 음수가 나오는 것을 확인할 수 있어요!

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(fashine_train_x[0], cmap='gray')
test_data = prepare_test_image(fashine_train_x[0])

In [ ]:
model.predict(test_data)

In [ ]:
tf.math.softmax(model.predict(test_data)) # scores to probabilities

### 학습시켰던 클래스를 가지고 있는 mnist 영상에 대해서는 어떨까요?
### 해당 클래스의 스코어 값은 양수로 표현됩니다.

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_test[0,:,:,0], cmap='gray')

In [ ]:
x_test[0:1,:,:].shape

In [ ]:
model.predict(x_test[0:1,:,:])

In [ ]:
tf.math.softmax(model.predict(x_test[0:1,:,:])) # scores to probabilities